In [1]:
import collections
import pygame
import numpy as np

""" Breadth First Search Algorithm. """
class Queue:
    def __init__(self):
        self.elements = collections.deque()
    
    def empty(self):
        return len(self.elements) == 0
    
    def put(self, x):
        self.elements.append(x)
    
    def get(self):
        return self.elements.popleft()


class SquareGraph:
    def __init__(self, cols, rows):
        self.cols = cols
        self.rows = rows
        self.graph = [[0 for x in range(cols)] for y in range(rows)]
        self.walls = []
    
    def construct_graph(self, start=None, goal=None, path=None):
        if start != None:
            self.graph[start[0]][start[1]] = 's'
        if goal != None:
            self.graph[goal[0]][goal[1]] = 'g'
        if path != None:
            steps = 1
            for step in path:
                self.graph[step[0]][step[1]] = steps
                steps += 1
    
    def add_wall(self, new_wall):
        self.walls.append(new_wall)
        self.graph[new_wall[0]][new_wall[1]] = 'w'
    
    def remove_wall(self, removed_wall):
        self.walls.remove(removed_wall)
        self.graph[removed_wall[0]][removed_wall[1]] = 0
    
    def in_graph(self, id):
        (x, y) = id
        return 0 <= x < self.cols and 0 <= y < self.rows
    
    def not_wall(self, id):
        return id not in self.walls
        
    def neighbors(self, id):
        (x, y) = id 
        moves = [(x, y+1), (x+1, y), (x, y-1), (x-1, y)]
        # moves = [(x, y+1), (x+1, y), (x, y-1), (x-1, y), (x+1, y+1), (x+1, y-1), (x-1, y-1), (x-1, y+1)]   # Diagonals
        
        if (x + y) % 2 == 0: moves.reverse()               
        moves = filter(self.in_graph, moves)
        moves = filter(self.not_wall, moves)
        return moves

    
def construct_path(start, goal, parents):
    current = goal
    path = []
    
    while current != start:
        path.append(current)
        current = parents[current]
    path.append(start)
    path.reverse()
    
    return path
    

def breadth_first_search(graph, start, goal):
    frontier = Queue()
    frontier.put(start)
    came_from = {}
    came_from[start] = None 

    while not frontier.empty():
        current = frontier.get()
        if current == goal:
            break
        for next in graph.neighbors(current):
            if next not in came_from:
                frontier.put(next)
                came_from[next] = current
    
    return came_from




""" GAME FUNCTIONS. """
# Allow for user to quit out of menu and game
def quit_game():
    pygame.quit()
    quit()
    
    
# Formats text display    
def text_objects(text, font):
    textSurface = font.render(text, True, WHITE)
    return textSurface, textSurface.get_rect()   


# Creates button controls
def button(msg, x, y, w, h, ic, ac, action=None):
    mouse = pygame.mouse.get_pos()
    click = pygame.mouse.get_pressed()  
    if x+w > mouse[0] > x and y+h > mouse[1] > y:
        pygame.draw.rect(gameDisplay, ac, (x,y,w,h))
        if click[0] == 1 and action != None:
            action()
    else:
        pygame.draw.rect(gameDisplay, ic, (x,y,w,h))
    smallText = pygame.font.SysFont("comicsansms", 20)
    textSurf, textRect = text_objects(msg, smallText)
    textRect.center = ( (x+(w/2)), (y+(h/2)) )
    gameDisplay.blit(textSurf, textRect)


# Returns the node that was clicked by the mouse
def clicked_node():
    pos = pygame.mouse.get_pos()
    row = pos[1] // (node_height + node_margin)
    col = pos[0] // (node_width + node_margin)    
    return row, col
    

# Update game display 
def grids_update():
    gameDisplay.fill(BLACK)
    for i in range(g.rows):
        for j in range(g.cols):
            color = WHITE
            if g.graph[i][j] == 'w':
                color = GREY
            elif g.graph[i][j] == 's':
                color = GREEN
            elif g.graph[i][j] == 'g':
                color = RED
            elif g.graph[i][j] > 0:
                color = BLUE
            pygame.draw.rect(gameDisplay, 
                             color, 
                             [(node_margin + node_width) * j + node_margin,
                              (node_margin + node_height) * i + node_margin,
                              node_width,
                              node_height])
    clock.tick(60)
    pygame.display.update()
    

# Starting menu for game
def game_intro():
    intro = True
    while intro:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                quit_game()
        gameDisplay.fill(BLACK)
        largeText = pygame.font.SysFont('comicsansms', 30)
        TextSurf, TextRect = text_objects("Pathfinder Visualization", largeText)
        TextRect.center = ((display_width/2), (display_height/2))
        gameDisplay.blit(TextSurf, TextRect)
        button("Start", 30,200,50,20, GREEN, BRIGHT_GREEN, game_loop)
        button("Quit", 130,200,50,20, RED, BRIGHT_RED, quit_game)
        pygame.display.update()
        clock.tick(15)

    
# Main game         
def game_loop():
    gameExit = False
    while not gameExit:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:               
                quit_game()
                
            elif event.type == pygame.MOUSEBUTTONDOWN:    # Obstacle locations
                g_row, g_col = clicked_node()
                if (g.graph[g_row][g_col] != 's' and g.graph[g_row][g_col] != 'g' and g.graph[g_row][g_col] != 'w'):    # Prevent wall replacing start/goal 
                    g.add_wall((g_row,g_col))
                elif (g.graph[g_row][g_col] == 'w'):
                    g.remove_wall((g_row,g_col))
                    
            if event.type == pygame.KEYDOWN:              # Start pathfinding
                if event.key == pygame.K_RETURN:  
                    parents = breadth_first_search(g, start, goal)
                    path = construct_path(start, goal, parents)
                    g.construct_graph(path=path)
            
        grids_update()
        


""" SET UP. """
# Colors  
BLACK = (0,0,0)           # background
WHITE = (255,255,255)     # nodes
DARK_GREY = (200,200,200) # boarder ('b')
GREY = (100,100,100)      # walls ('w')
GREEN = (0,255,0)         # start ('s')
RED = (255,0,0)           # goal ('g')
BLUE = (0,0,255)          # path (1)
BRIGHT_RED = (255,0,0)
BRIGHT_GREEN = (0,255,0)

# Node and grid dimensions 
node_width = 20
node_height = 20
node_margin = 5

# Graph for maze ######
grid_width = 21
grid_height = 21
g = SquareGraph(grid_width, grid_height)
start = (0,10)
goal = (20,10)
g.construct_graph(start=start, goal=goal)

# Game display
pygame.init()
display_width = (node_width*grid_width) + (node_margin*(1+node_width))
display_height = (node_height*grid_height) + (node_margin*(1+node_height))
gameDisplay = pygame.display.set_mode((display_width, display_height))
pygame.display.set_caption('Pathfinder')
gameDisplay.fill(BLACK)
clock = pygame.time.Clock()
        

game_intro()
game_loop()
quit_game()     

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


error: display Surface quit

In [1]:
import collections
import pygame
import numpy as np

""" Breadth First Search Algorithm. """
class Queue:
    def __init__(self):
        self.elements = collections.deque()
    
    def empty(self):
        return len(self.elements) == 0
    
    def put(self, x):
        self.elements.append(x)
    
    def get(self):
        return self.elements.popleft()


class SquareGraph:
    def __init__(self, cols, rows):
        self.cols = cols
        self.rows = rows
        self.graph = [[0 for x in range(cols)] for y in range(rows)]
        self.walls = []
    
    def construct_graph(self, start=None, goal=None, path=None):
        for item in self.walls:
            self.graph[item[0]][item[1]] = 'w'
        if start != None:
            self.graph[start[0]][start[1]] = 's'
        if goal != None:
            self.graph[goal[0]][goal[1]] = 'g'
        if path != None:
            steps = 1
            for step in path:
                self.graph[step[0]][step[1]] = steps
                steps += 1
    
    def add_wall(self, new_wall):
        self.walls.append(new_wall)
        self.graph[new_wall[0]][new_wall[1]] = 'w'
    
    def remove_wall(self, removed_wall):
        self.walls.remove(removed_wall)
        self.graph[removed_wall[0]][removed_wall[1]] = 0
    
    def in_graph(self, id):
        (x, y) = id
        return 0 <= x < self.cols and 0 <= y < self.rows
    
    def not_wall(self, id):
        return id not in self.walls
        
    def neighbors(self, id):
        (x, y) = id 
        moves = [(x, y+1), (x+1, y), (x, y-1), (x-1, y)] 
        if (x + y) % 2 == 0: moves.reverse()               
        moves = filter(self.in_graph, moves)
        moves = filter(self.not_wall, moves)
        return moves

    
def construct_path(start, goal, parents):
    current = goal
    path = []
    while current != start:
        path.append(current)
        current = parents[current]
    path.append(start)
    path.reverse()
    return path
    

def breadth_first_search(graph, start, goal):
    frontier = Queue()
    frontier.put(start)
    came_from = {}
    came_from[start] = None 

    while not frontier.empty():
        current = frontier.get()
        if current == goal:
            break
        for next in graph.neighbors(current):
            if next not in came_from:
                frontier.put(next)
                came_from[next] = current
    
    return came_from



""" Dijkstra's Algorithm. """
import heapq

# Implements movement costs for SquareGraph
class WeightedGraph(SquareGraph):
    def __init__(self, cols, rows):
        super().__init__(cols, rows)
        self.weights = {}
    
    def cost(self, from_node, to_node):
        return self.weights.get(to_node, 1)

    
class PriorityQueue():
    def __init__(self):
        self.elements = []
    
    def empty(self):
        return len(self.elements) == 0
    
    def put(self, item, priority):
        heapq.heappush(self.elements, (priority, item))
        
    def get(self):
        return heapq.heappop(self.elements)[1]
    
    
def dijkstras(graph, start, goal):
    frontier = PriorityQueue()
    frontier.put(start, 0)
    came_from = {}
    cost_so_far = {}
    came_from[start] = None
    cost_so_far[start] = 0
    
    while not frontier.empty():
        current = frontier.get()
        if current == goal:
            break
        for next in graph.neighbors(current):
            new_cost = cost_so_far[current] + graph.cost(current, next)
            if next not in cost_so_far or new_cost < cost_so_far[next]:
                cost_so_far[next] = new_cost
                priority = new_cost
                frontier.put(next, priority)
                came_from[next] = current
                
    return came_from, cost_so_far
    

""" GAME FUNCTIONS. """
# Allow for user to quit out of menu and game
def quit_game():
    pygame.quit()
    quit()
    
    
# Formats text display    
def text_objects(text, font):
    textSurface = font.render(text, True, WHITE)
    return textSurface, textSurface.get_rect()   


# Creates button controls
def button(msg, x, y, w, h, ic, ac, action=None):
    mouse = pygame.mouse.get_pos()
    click = pygame.mouse.get_pressed()  
    if x+w > mouse[0] > x and y+h > mouse[1] > y:
        pygame.draw.rect(gameDisplay, ac, (x,y,w,h))
        if click[0] == 1 and action != None:
            action()
    else:
        pygame.draw.rect(gameDisplay, ic, (x,y,w,h))
    smallText = pygame.font.SysFont("comicsansms", 20)
    textSurf, textRect = text_objects(msg, smallText)
    textRect.center = ( (x+(w/2)), (y+(h/2)) )
    gameDisplay.blit(textSurf, textRect)


# Returns the node that was clicked by the mouse
def clicked_node():
    pos = pygame.mouse.get_pos()
    row = pos[1] // (node_height + node_margin)
    col = pos[0] // (node_width + node_margin)    
    return row, col
    

# Update game display 
def grids_update():
    gameDisplay.fill(BLACK)
    for i in range(g.rows):
        for j in range(g.cols):
            color = WHITE
            if g.graph[i][j] == 'w':
                color = GREY
            elif g.graph[i][j] == 's':
                color = GREEN
            elif g.graph[i][j] == 'g':
                color = RED
            elif g.graph[i][j] > 0:
                color = BLUE
            pygame.draw.rect(gameDisplay, 
                             color, 
                             [(node_margin + node_width) * j + node_margin,
                              (node_margin + node_height) * i + node_margin,
                              node_width,
                              node_height])
    clock.tick(60)
    pygame.display.update()
    

# Starting menu for game
def game_intro():
    intro = True
    while intro:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                quit_game()
        gameDisplay.fill(BLACK)
        largeText = pygame.font.SysFont('comicsansms', 30)
        TextSurf, TextRect = text_objects("Pathfinder Visualization", largeText)
        TextRect.center = ((display_width/2), (display_height/2))
        gameDisplay.blit(TextSurf, TextRect)
        button("Start", 30,200,50,20, GREEN, BRIGHT_GREEN, game_loop)
        button("Quit", 130,200,50,20, RED, BRIGHT_RED, quit_game)
        pygame.display.update()
        clock.tick(15)

    
# Main game         
def game_loop():
    gameExit = False
    while not gameExit:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:               
                quit_game()
                
            elif event.type == pygame.MOUSEBUTTONDOWN:    # Obstacle locations
                g_row, g_col = clicked_node()
                if (g.graph[g_row][g_col] != 's' and g.graph[g_row][g_col] != 'g' and g.graph[g_row][g_col] != 'w'):    # Prevent wall replacing start/goal 
                    g.add_wall((g_row,g_col))
                elif (g.graph[g_row][g_col] == 'w'):
                    g.remove_wall((g_row,g_col))
                    
            if event.type == pygame.KEYDOWN:              # Start pathfinding
                if event.key == pygame.K_RETURN:  
                    # parents = breadth_first_search(g, start, goal)
                    parents, cost_so_far = dijkstras(g, start, goal)
                    path = construct_path(start, goal, parents)
                    g.construct_graph(path=path)
            
        grids_update()
        


""" SET UP. """
# Colors  
BLACK = (0,0,0)           # background
WHITE = (255,255,255)     # nodes
DARK_GREY = (200,200,200) # boarder ('b')
GREY = (100,100,100)      # walls ('w')
GREEN = (0,255,0)         # start ('s')
RED = (255,0,0)           # goal ('g')
BLUE = (0,0,255)          # path (1)
BRIGHT_RED = (255,0,0)
BRIGHT_GREEN = (0,255,0)

# Node and grid dimensions 
node_width = 20
node_height = 20
node_margin = 5

# Graph for maze ######
grid_width = 11
grid_height = 11
# g = SquareGraph(grid_width, grid_height)
g = WeightedGraph(grid_width, grid_height)
start = (0, int((grid_width-1)/2))
goal = (grid_height-1, int((grid_width-1)/2))
g.construct_graph(start=start, goal=goal)

# Game display
pygame.init()
display_width = (node_width*grid_width) + (node_margin*(1+grid_width))
display_height = (node_height*grid_height) + (node_margin*(1+grid_height))
gameDisplay = pygame.display.set_mode((display_width, display_height))
pygame.display.set_caption('Pathfinder')
gameDisplay.fill(BLACK)
clock = pygame.time.Clock()
        

game_intro()
game_loop()
quit_game()     

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


error: display Surface quit